In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps.datasets
# Import API key
from api_keys import g_key

In [2]:
city_df = pd.read_csv("output_data/cities.csv")
city_df.head()



,City_id,City,Cloudiness,Country,Date,Humidity,latitude,longitude,Max temp,Wind Speed
0,0,Busselton,35,AU,1584490087,92,-33.65,115.33,287.04,7.81
1,1,Lagoa,0,PT,1584490087,80,39.05,-27.98,290.14,7.13
2,2,Butaritari,0,KI,1584490087,75,3.07,172.79,301.68,5.67
3,3,Ushuaia,40,AR,1584490087,41,-54.80,-68.30,287.15,7.20
4,4,Kahului,75,US,1584490087,69,20.89,-156.47,297.15,13.40


In [3]:
# Configure gmaps with API key
gmaps.configure(api_key= g_key)
humidity_rate = city_df["Humidity"].astype(float)

# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["latitude", "longitude"]]


# Create a humidity Heatmap layer
fig = gmaps.figure()




heat_layer = gmaps.heatmap_layer(locations, weights = humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

hotel_df = city_df.loc[(city_df["Max temp"]>300) & (city_df["Max temp"]<308) & (city_df['Wind Speed'] < 5) & (city_df['Cloudiness'] == 0), :]
 
hotel_df.head(90) 

#city_df['Wind Speed'].max()



,City_id,City,Cloudiness,Country,Date,Humidity,latitude,longitude,Max temp,Wind Speed
56,56,Lázaro Cárdenas,0,MX,1584490099,67,17.96,-102.20,300.18,3.47
164,164,Alice Springs,0,AU,1584489840,32,-23.70,133.88,300.15,4.60
172,172,Auki,0,SB,1584490125,73,-8.77,160.70,302.00,1.90
248,248,Yulara,0,AU,1584490141,28,-25.24,130.99,301.15,4.60
316,316,Caravelas,0,BR,1584490156,82,-17.71,-39.25,300.43,4.76
318,318,Kralendijk,0,BQ,1584490156,69,12.15,-68.27,300.15,4.60
360,360,Tomatlán,0,MX,1584490166,51,19.93,-105.25,300.67,3.08
429,429,Huitziltepec,0,MX,1584490181,26,17.82,-99.47,307.59,0.45
436,436,Sorong,0,ID,1584489907,67,-0.88,131.25,301.60,1.13
438,438,Kununurra,0,AU,1584490183,70,-15.77,128.73,303.15,1.50


In [5]:
hotel_df['Hotel Name'] = ""

hotel_df

C:\Users\kames\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City_id,City,Cloudiness,Country,Date,Humidity,latitude,longitude,Max temp,Wind Speed,Hotel Name
56,56,Lázaro Cárdenas,0,MX,1584490099,67,17.96,-102.20,300.18,3.47,
164,164,Alice Springs,0,AU,1584489840,32,-23.70,133.88,300.15,4.60,
172,172,Auki,0,SB,1584490125,73,-8.77,160.70,302.00,1.90,
248,248,Yulara,0,AU,1584490141,28,-25.24,130.99,301.15,4.60,
316,316,Caravelas,0,BR,1584490156,82,-17.71,-39.25,300.43,4.76,
318,318,Kralendijk,0,BQ,1584490156,69,12.15,-68.27,300.15,4.60,
360,360,Tomatlán,0,MX,1584490166,51,19.93,-105.25,300.67,3.08,
429,429,Huitziltepec,0,MX,1584490181,26,17.82,-99.47,307.59,0.45,
436,436,Sorong,0,ID,1584489907,67,-0.88,131.25,301.60,1.13,
438,438,Kununurra,0,AU,1584490183,70,-15.77,128.73,303.15,1.50,


In [6]:
params = {
     
    
      "type": "hotel",
      "keyword": "Hotel",
    "radius": 5000,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    
    # get hotel coordinated from df
    lat = row["latitude"]
    lon = row["longitude"]
    # add keyword to params dict
    params['location'] = f"{lat},{lon}"

 # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel for this city {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 56: Lázaro Cárdenas.
Closest hotel for this city Lázaro Cárdenas is Hotel de Casablanca.
------------
Retrieving Results for Index 164: Alice Springs.


C:\Users\kames\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel for this city Alice Springs is DoubleTree by Hilton Hotel Alice Springs.
------------
Retrieving Results for Index 172: Auki.
Closest hotel for this city Auki is Auki Lodge.
------------
Retrieving Results for Index 248: Yulara.
Closest hotel for this city Yulara is Sails in the Desert.
------------
Retrieving Results for Index 316: Caravelas.
Closest hotel for this city Caravelas is Pousada Liberdade.
------------
Retrieving Results for Index 318: Kralendijk.
Closest hotel for this city Kralendijk is Delfins Beach Resort Bonaire.
------------
Retrieving Results for Index 360: Tomatlán.
Closest hotel for this city Tomatlán is Hotel Misión San Miguel.
------------
Retrieving Results for Index 429: Huitziltepec.
Missing field/result... skipping.
------------
Retrieving Results for Index 436: Sorong.
Closest hotel for this city Sorong is Swiss-Belhotel Sorong.
------------
Retrieving Results for Index 438: Kununurra.
Closest hotel for this city Kununurra is Hotel Kununurra.


In [7]:
hotel_df

,City_id,City,Cloudiness,Country,Date,Humidity,latitude,longitude,Max temp,Wind Speed,Hotel Name
56,56,Lázaro Cárdenas,0,MX,1584490099,67,17.96,-102.20,300.18,3.47,Hotel de Casablanca
164,164,Alice Springs,0,AU,1584489840,32,-23.70,133.88,300.15,4.60,DoubleTree by Hilton Hotel Alice Springs
172,172,Auki,0,SB,1584490125,73,-8.77,160.70,302.00,1.90,Auki Lodge
248,248,Yulara,0,AU,1584490141,28,-25.24,130.99,301.15,4.60,Sails in the Desert
316,316,Caravelas,0,BR,1584490156,82,-17.71,-39.25,300.43,4.76,Pousada Liberdade
318,318,Kralendijk,0,BQ,1584490156,69,12.15,-68.27,300.15,4.60,Delfins Beach Resort Bonaire
360,360,Tomatlán,0,MX,1584490166,51,19.93,-105.25,300.67,3.08,Hotel Misión San Miguel
429,429,Huitziltepec,0,MX,1584490181,26,17.82,-99.47,307.59,0.45,
436,436,Sorong,0,ID,1584489907,67,-0.88,131.25,301.60,1.13,Swiss-Belhotel Sorong
438,438,Kununurra,0,AU,1584490183,70,-15.77,128.73,303.15,1.50,Hotel Kununurra


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))